# Comparing Old MEP, CoR and ECB with current lobbyist

In [ ]:
#Get dataset of all MEP
!wget 'http://www.europarl.europa.eu/RegData/publications/lmeps/1979/0001/EP-PE_LMEPS(1979)0001_XL.xls'

In [5]:
from scripts import import_MEP
df_MEP = import_MEP('/mnt/2To/jupyter_data/FTM/Revolving_doors/EP-PE_LMEPS(1979)0001_XL.xls')


In [ ]:
# Get dataset of accredited lobbyist
!wget https://transparency-register.europa.eu/odplastaccreditedxml_en 
# Turn xml in json
!python -m xmljson '/mnt/2To/jupyter_data/FTM/Revolving_doors/odplastaccreditedxml_en.xml' -o 'accredited.json'

In [6]:
from scripts import import_accredited
df_accredited=import_accredited('accredited.json')

In [7]:
# Find intersection
import pandas as pd
merged = pd.merge(df_MEP, df_accredited, how='inner', on=['fullname', 'fullname'])

In [8]:
# Old MEP that are currently registered in the accredited database
merged

,NOM,État membre,Parti politique national*,Groupe politique*,Mandats,fullname,firstName,lastName,orgIdentificationCode,orgName,accreditationStartDate,accreditationEndDate,orgNameInLatinAlphabet
0,BÖGE Reimer,DE,CDU,PPE,25.07.1989,böge reimer,Reimer,Böge,55454323168-03,Arbeitsgemeinschaft Deutscher Tierzüchter e. V.,27/04/2024,31/12/2025,NaN
1,BONETTI Andrea,IT,DC,PPE,25.07.1989-18.07.1994,bonetti andrea,Andrea,BONETTI,974918817911-97,Chiesi Farmaceutici SpA,10/06/2024,10/06/2026,NaN
2,CHATZIMARKAKIS Jorgo,DE,FDP,ALDE,20.07.2004-30.06.2014,chatzimarkakis jorgo,Jorgo,Chatzimarkakis,77659588648-75,Hydrogen Europe,20/09/2023,19/09/2024,NaN
3,DE ANGELIS Francesco,IT,PD,S&D,14.07.2009-30.06.2014,de angelis francesco,Francesco,De Angelis,718251720163-42,Flint Europe,06/10/2023,04/10/2024,NaN
4,DELL'ALBA Gianfranco,IT,Lista Bonino,NI,19.07.1994-19.07.2004,dell'alba gianfranco,Gianfranco,DELL'ALBA,150086650957-26,GFDA Consulting srl,21/09/2023,20/09/2024,NaN
5,MAURO Mario,IT,PDL,PPE,20.07.1999-14.03.2013,mauro mario,Mario,MAURO,736538940689-06,MESEURO Srl,05/01/2024,03/01/2025,NaN
6,SIMON Peter,DE,SPD,S&D,14.07.2009,simon peter,Peter,SIMON,8765978796-80,European Savings and Retail Banking Group,04/06/2024,01/06/2025,NaN
7,VIOLA Vincenzo,IT,Patto Segni,PPE,18.09.1995-19.07.1999,viola vincenzo,Vincenzo,Viola,32591134448-30,European Producers Union of Renewable Ethanol,09/07/2024,09/07/2025,NaN
8,VIOLA Vincenzo,IT,Patto Segni,PPE,18.09.1995-19.07.1999,viola vincenzo,Vincenzo,VIOLA,85076002321-31,The EU Vegetable Oil and Proteinmeal Industry,09/11/2023,08/11/2024,NaN


We found 9 old MEP currently registered as accredited lobbyist

## Adding names from ECB pdf
We run NLP name detection to extract all names of the ECB structure available online

In [2]:
from scripts import import_ECB
peoples_ECB = import_ECB('/mnt/2To/jupyter_data/FTM/Revolving_doors/ecb.ECB_managers.en.pdf') #Archive PDF of ECB structure 

  0%|          | 0/20 [00:00<?, ?it/s]

In [3]:
# Generate double entries for to account for error in the order of the name for ECB
from scripts import create_variation
peoples_ECB = create_variation(peoples_ECB)

  0%|          | 0/271 [00:00<?, ?it/s]

In [9]:
df_accredited[df_accredited['fullname'].isin(peoples_ECB)]

,firstName,lastName,orgIdentificationCode,orgName,accreditationStartDate,accreditationEndDate,orgNameInLatinAlphabet,fullname


We find no intersection between old ECB staff and accredited lobbyist

# Find lobbyist in outgoing CoR 

In [1]:
from selenium_CoR import get_outgoing_CoR

CoR_people = get_outgoing_CoR() #Scraper getting data from https://memberspage.cor.europa.eu/members

Element text is: Outgoing


In [38]:
df_accredited[df_accredited['fullname'].isin(CoR_people)]

,firstName,lastName,orgIdentificationCode,orgName,accreditationStartDate,accreditationEndDate,orgNameInLatinAlphabet,fullname
871,Guido,Castelli,098690717154-62,Università Cattolica del Sacro Cuore,24/05/2024,24/05/2026,NaN,castelli guido


Only one accredited lobbyist in the outgoing CoR webpage

## Archive of staffers
should use https://github.com/rliebz/whoswho ?

In [11]:
# creating a pdf reader object
import PyPDF2
import spacy
from tqdm.notebook import tqdm
import glob
for pdf in tqdm(glob.glob("/mnt/2To/jupyter_data/FTM/Revolving_doors/PDF_ARCHIVE_EU_WHOISWHO_202012.EN/PDF_ARCHIVE_EU_WHOISWHO_202012.EN/*.pdf")):
    reader = PyPDF2.PdfReader(pdf)
    pages = [32, ]
    NER = spacy.load("en_core_web_trf")
    peoples = []
    for page in reader.pages[:32]:
        text1= NER(" ".join(page.extract_text().split()))#remove double spaces and rejoin text
        for word in text1.ents:
            print(word.text,word.label_)
            if (word.label_ == 'PERSON') and (len(word.text.split())>1):
                peoples.append(word.text)

  0%|          | 0/15 [00:00<?, ?it/s]

EUROPEAN UNION EU ORG
16/12/2020 DATE
the Publications Office ORG
European Union ORG
2020 DATE
1 CARDINAL
20160818 DATE
EU ORG
the Publications Office ORG
European Union ORG
2020 DATE
12 CARDINAL
rue Alcide De Gasperi FAC
16/12/2020 – 3 DATE
BRUSSELS 4 – 16/12/2020 – DATE
BRUSSELS EUROPEAN NORP
16/12/2020 – 5 DATE
LUXEMBOURG GPE
Kirchberg Plateau LOC
EUROPEAN NORP
LUXEMBOURG GPE
16/12/2020 – 7 DATE
European Court of Auditors ORG
10 CARDINAL
I CARDINAL
II CARDINAL
III CARDINAL
IV CARDINAL
V CARDINAL
Union ORG
Audit Quality Control Committee ORG
Administrative Committee ORG
Internal Audit Committee ORG
Digital Steering Committee ORG
The Directorate of Human Resources, Finance and General Services ORG
The Directorate of Information, Workplace and Innovation ORG
The Directorate of Translation, Language Services and Publication ORG
13 8 – 16/12/2020 DATE
European Court of Auditors ORG
12 CARDINAL
rue Alcide De Gasperi FAC
Luxembourg GPE
LUXEMBOURG GPE
Klaus-Heiner LEHNE PERSON
Thomas ARNTZ 

In [13]:
import pickle
with open('peopls_archive.pkl', 'wb') as f:
    pickle.dump(peoples, f)